In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
%pip install unbabel-comet>=2.2.0 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
%pip install --upgrade comet-ml -q


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
from datasets import load_dataset
from transformers import AutoModel

AllocationException: VM could not be allocated

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/XCOMET-XL")
model = load_from_checkpoint(model_path)
model = model.to(device)

Fetching 5 files: 100%|██████████| 5/5 [03:32<00:00, 42.46s/it]
Encoder model frozen.
/home/jupyter/.local/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [8]:
data = [
    {
        # Source, текст, который надо перевести, src
        "src": "В понедельник ученые из Медицинской школы Стэнфордского университета объявили об изобретении нового диагностического инструмента, который может сортировать клетки по их типу; это маленький чип, который можно напечатать, используя стандартный струйный принтер примерно за 1 цент США.", 
        
        # Machine Translation
        "mt": "On Monday, scientists from Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by their type; it's a small chip that can be printed using a standard inkjet printer for about 1 cent USD.", 
        
        # Эталонный перевод, en
        "ref": "On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each." 
    },
    
    {
    
        # Source, текст, который надо перевести, src
        "src": "В понедельник ученые из Медицинской школы Стэнфордского университета объявили об изобретении нового диагностического инструмента, который может сортировать клетки по их типу; это маленький чип, который можно напечатать, используя стандартный струйный принтер примерно за 1 цент США.", 
        
        # Machine Translation
        "mt": "On Monday, scientists from Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by their type; it's a small chip that can be printed using a standard inkjet printer for about 1 cent USD.", 
        
        # Эталонный перевод, en
        "ref": "On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each." 
    }
]

model_output = model.predict(data, batch_size=8, gpus=1)
# Segment-level scores
print (model_output.scores)

# System-level score
print(model_output.system_score)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  1.14s/it]


[0.9696990251541138, 0.9696990251541138]
0.9696990251541138


In [ ]:
model_path_kiwi = download_model("Unbabel/wmt23-cometkiwi-da-xxl")
model_kiwi = load_from_checkpoint(model_path_kiwi)
model_kiwi = model_kiwi.to(device)

In [ ]:
from typing import Dict, List
import torch
# from comet import download_model, load_from_checkpoint

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model_path_xcomet = download_model("Unbabel/XCOMET-XL")
# model_xcomet = load_from_checkpoint(model_path_xcomet)
# model_xcomet = model_xcomet.to(device)

# model_path_kiwi = download_model("Unbabel/wmt23-cometkiwi-da-xxl")
# model_kiwi = load_from_checkpoint(model_path_kiwi)
# model_kiwi = model_kiwi.to(device)

# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("cointegrated/roberta-large-cola-krishna2020")
# model_fluency = AutoModelForSequenceClassification.from_pretrained("cointegrated/roberta-large-cola-krishna2020")
# model_fluency = model_fluency.to(device)

def xcomet(data: List[Dict[str, str]]) -> List[float]:
    
    '''Format
    data = [
    {
        # Source, текст, который надо перевести, src
        "src": "В понедельник", 
        
        # Machine Translation
        "mt": "On Monday", 
        
        # Эталонный перевод, en
        "ref": "On Monday" 
    }'''
    
    
    xcomet_metric = model_xcomet.predict(data, batch_size=8, gpus=1)
    return xcomet_metric.scores


def kiwi(data: List[Dict[str, str]]) -> List[float]:
    
    '''Format
    data = [
    {
        # source original text, ru
        "src": "В понедельник",
        
        # machine translation, en
        "mt": "On Monday"
    }'''
    
    
    kiwi_metric = model_kiwi.predict(data, batch_size=8, gpus=1)
    return kiwi_metric


def fluency(data: str) -> float:
    return float(torch.softmax(model_fluency(**tokenizer(data, return_tensors="pt").to(device)).logits, dim = -1)[0][0].detach().cpu())

In [ ]:
data = [{
    "src": "В понедельник ученые из Медицинской школы Стэнфордского университета объявили об изобретении нового диагностического инструмента, который может сортировать клетки по их типу; это маленький чип, который можно напечатать, используя стандартный струйный принтер примерно за 1 цент США.", 
    "mt": "On Monday, scientists from Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by their type; it's a small chip that can be printed using a standard inkjet printer for about 1 cent USD."}]

kiwi(data)